In [2]:
import pandas as pd

In [3]:
turismo = pd.read_xml('https://www.esmadrid.com/opendata/turismo_v1_es.xml', xpath='//geoData')
turismo.head()

,address,zipcode,locality,country,latitude,longitude,subAdministrativeArea
0,"Ramón Sainz, 22",28025,None,Spain,40.387540,-3.737781,Madrid
1,"de Huertas, 71",28014,None,Spain,40.413246,-3.695318,Madrid
2,"de Serrano, 61",28006,None,Spain,40.431905,-3.687442,Madrid
3,"del Doctor Fourquet, 18",28012,None,Spain,40.407580,-3.697474,Madrid
4,"de Serrano, 56",28004,None,Spain,40.428312,-3.687092,Madrid


In [4]:
turismo.shape

(901, 7)

In [5]:
grouped_tourism = turismo.groupby('zipcode')['address'].count()

In [6]:
grouped_tourism

zipcode
-         3
28001    34
28002     9
28003    22
28004    85
         ..
28911     1
28914     1
28931     1
28939     2
45004     1
Name: address, Length: 84, dtype: int64

In [7]:
total_foreign_tourists = 6000000
#only foreign, not counting national tourists
# Calculate the total number of attractions
total_attractions = grouped_tourism.sum()

In [8]:
total_attractions

888

In [9]:
attraction_proportions = grouped_tourism / total_attractions

# Calculate the weights
weights = attraction_proportions * total_foreign_tourists

# Add the weights to the Series
data = grouped_tourism.to_frame(name='counts').assign(weights=weights)

In [10]:
data

,counts,weights
zipcode,,
-,3,20270.270270
28001,34,229729.729730
28002,9,60810.810811
28003,22,148648.648649
28004,85,574324.324324
...,...,...
28911,1,6756.756757
28914,1,6756.756757
28931,1,6756.756757


In [11]:
data.reset_index(inplace=True)

In [12]:
data.drop(0, inplace=True)
data

,zipcode,counts,weights
1,28001,34,229729.729730
2,28002,9,60810.810811
3,28003,22,148648.648649
4,28004,85,574324.324324
5,28005,62,418918.918919
...,...,...,...
79,28911,1,6756.756757
80,28914,1,6756.756757
81,28931,1,6756.756757
82,28939,2,13513.513514


In [13]:
data['zipcode'] = data['zipcode'].astype(int)

In [14]:
data.to_csv('tourists_data.csv')

In [12]:
google = pd.read_csv('google_public_data.csv')
google.drop('Unnamed: 0', axis=1, inplace=True)
google.head()

,result.business_status,result.delivery,result.dine_in,result.name,result.place_id,result.rating,result.reservable,result.serves_beer,result.serves_breakfast,result.serves_brunch,...,night_club,zip_code,price_level,success,distrito,total_population,restaurants_per_district_area,share_year_budget_rest_per_household,share_local_pop,share_of_surface_in_KM2
0,OPERATIONAL,True,1,restaurante la dehesa,ChIJe8y4CKAuQg0RC4YervOOVLg,4.0,True,1,1,1,...,0,28050,2.0,1,HORTALEZA,195017,37.688442,17.863179,200.017436,0.026533
1,OPERATIONAL,True,1,casa corrochano,ChIJjWIe0PMrQg0RXJxYC8mlRcM,4.1,True,1,1,1,...,0,28050,1.0,1,HORTALEZA,195017,37.688442,17.863179,200.017436,0.026533
2,OPERATIONAL,Unknown,1,la mano de pablo,ChIJmw6_CKAuQg0RY9Kcr3tR5OQ,4.4,True,1,0,1,...,0,28050,2.0,1,HORTALEZA,195017,37.688442,17.863179,200.017436,0.026533
3,OPERATIONAL,False,1,el oso,ChIJsxAsqggsQg0Rz1mITdkhDaY,4.1,True,1,1,1,...,0,28050,3.0,0,HORTALEZA,195017,37.688442,17.863179,200.017436,0.026533
4,OPERATIONAL,Unknown,1,domino,ChIJjd0Nhu0rQg0R1Q0k5TsewKg,3.7,True,1,1,1,...,0,28050,1.0,1,HORTALEZA,195017,37.688442,17.863179,200.017436,0.026533


In [13]:
google.shape

(5233, 39)

In [14]:
google_w = pd.merge(google, data, left_on='zip_code', right_on='zipcode')

In [22]:
tourists_df = google_w.groupby('zipcode')['weights'].mean()

In [21]:
num_restaurants_by_zipcode = google_w.groupby('zipcode')['result.name'].count().reset_index()

In [30]:
num_restaurants_by_zipcode

,zipcode,result.name
0,28001,78
1,28002,122
2,28003,99
3,28004,102
4,28005,120
5,28006,103
6,28007,118
7,28008,98
8,28009,108
9,28010,102


In [28]:
num_restaurants_by_zipcode = num_restaurants_by_zipcode.rename(columns={'restaurant_name': 'num_restaurants'})

In [31]:
google_tourists  = pd.merge(google_w, num_restaurants_by_zipcode, on='zipcode')

In [32]:
google_tourists

,result.business_status,result.delivery,result.dine_in,result.name_x,result.place_id,result.rating,result.reservable,result.serves_beer,result.serves_breakfast,result.serves_brunch,...,distrito,total_population,restaurants_per_district_area,share_year_budget_rest_per_household,share_local_pop,share_of_surface_in_KM2,zipcode,counts,weights,result.name_y
0,OPERATIONAL,True,1,restaurante la dehesa,ChIJe8y4CKAuQg0RC4YervOOVLg,4.0,True,1,1,1,...,HORTALEZA,195017,37.688442,17.863179,200.017436,0.026533,28050,3,878.378378,109
1,OPERATIONAL,True,1,casa corrochano,ChIJjWIe0PMrQg0RXJxYC8mlRcM,4.1,True,1,1,1,...,HORTALEZA,195017,37.688442,17.863179,200.017436,0.026533,28050,3,878.378378,109
2,OPERATIONAL,Unknown,1,la mano de pablo,ChIJmw6_CKAuQg0RY9Kcr3tR5OQ,4.4,True,1,0,1,...,HORTALEZA,195017,37.688442,17.863179,200.017436,0.026533,28050,3,878.378378,109
3,OPERATIONAL,False,1,el oso,ChIJsxAsqggsQg0Rz1mITdkhDaY,4.1,True,1,1,1,...,HORTALEZA,195017,37.688442,17.863179,200.017436,0.026533,28050,3,878.378378,109
4,OPERATIONAL,Unknown,1,domino,ChIJjd0Nhu0rQg0R1Q0k5TsewKg,3.7,True,1,1,1,...,HORTALEZA,195017,37.688442,17.863179,200.017436,0.026533,28050,3,878.378378,109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5128,OPERATIONAL,False,1,el rincon de paula ii,ChIJp_7UKJ0nQg0RnQKlxWxzKhM,4.3,True,1,1,1,...,VILLAVERDE,153829,23.701138,15.207933,321.146138,0.042192,28021,3,878.378378,107
5129,OPERATIONAL,False,1,casa domingo,ChIJHxgHwjEnQg0RpkM-ghqQ_Pk,4.4,True,1,1,1,...,VILLAVERDE,153829,23.701138,15.207933,321.146138,0.042192,28021,3,878.378378,107
5130,OPERATIONAL,False,1,bar la taskita,ChIJ8-lR1yQnQg0RwXHeD5PLr3I,3.0,Unknown,1,1,1,...,VILLAVERDE,153829,23.701138,15.207933,321.146138,0.042192,28021,3,878.378378,107
5131,OPERATIONAL,Unknown,1,restaurante lablanca,ChIJ1_rxn8cnQg0RkXO_edn5ygc,4.7,True,1,1,1,...,VILLAVERDE,153829,23.701138,15.207933,321.146138,0.042192,28021,3,878.378378,107


In [33]:
google_tourists['share_tourists_zipcode'] = google_tourists['weights'] / google_tourists['result.name_y']

In [35]:
google_tourists['share_tourists_zipcode'].sum()

238040.5405405405

In [36]:
google_tourists.to_csv('googlew.csv')

In [ ]:
pedestrians = pd.read_csv('peatones_limpios.csv')
pedestrians.head()

,Unnamed: 0,FECHA,PEATONES,DISTRITO,NOMBRE_VIAL,NÚMERO,CÓDIGO_POSTAL,OBSERVACIONES_DIRECCION,LATITUD,LONGITUD
0,0,2021-01-01 00:00:00,110.0,Centro,Calle Fuencarral,22,28004,Calle peatonal,"40,42200905","-3,700891722"
1,1,2021-01-01 00:00:00,111.0,Centro,Calle San Bernardo,36,28015,Acera Pares,"40,42380314","-3,707379398"
2,2,2021-01-01 00:00:00,15.0,Centro,Calle Hortaleza,18,28004,Acera Pares,"40,42107227","-3,70049043"
3,3,2021-01-01 00:00:00,44.0,Centro,Carrera de San Jerónimo,6,28014,Acera Pares,"40,41664098","-3,701246554"
4,4,2021-01-01 00:00:00,49.0,Centro,Calle Atocha,95,28012,Acera Impares,"40,41110888","-3,696471467"


In [ ]:
pedestrians.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
pedestrians.dtypes

FECHA                       object
PEATONES                   float64
DISTRITO                    object
NOMBRE_VIAL                 object
NÚMERO                       int64
CÓDIGO_POSTAL                int64
OBSERVACIONES_DIRECCION     object
LATITUD                     object
LONGITUD                    object
dtype: object

In [ ]:
pedestrians['FECHA'] = pd.to_datetime(pedestrians['FECHA'], infer_datetime_format=True)  

In [ ]:
pedestrians['FECHA']

0       2021-01-01 00:00:00
1       2021-01-01 00:00:00
2       2021-01-01 00:00:00
3       2021-01-01 00:00:00
4       2021-01-01 00:00:00
                ...        
82531   2021-06-30 23:00:00
82532   2021-06-30 23:00:00
82533   2021-06-30 23:00:00
82534   2021-06-30 23:00:00
82535   2021-06-30 23:00:00
Name: FECHA, Length: 82536, dtype: datetime64[ns]

In [ ]:
pedestrians.groupby('FECHA')['PEATONES'].sum()

FECHA
2021-01-01 00:00:00    1742.000
2021-01-01 01:00:00    1545.000
2021-01-01 02:00:00    1265.000
2021-01-01 03:00:00     882.000
2021-01-01 04:00:00     586.000
                         ...   
2021-12-06 19:00:00    5588.671
2021-12-06 20:00:00    5763.251
2021-12-06 21:00:00    5458.397
2021-12-06 22:00:00    5386.101
2021-12-06 23:00:00    4784.318
Name: PEATONES, Length: 4344, dtype: float64

In [ ]:
pedestrians['DIA'] = pedestrians['FECHA'].dt.date  # extract the date portion only
daily_counts = pedestrians.groupby('DIA')['PEATONES'].sum()

In [ ]:
daily_counts.nlargest(10)

DIA
2021-05-30    108231.612
2021-04-03    103701.191
2021-06-23     98006.167
2021-08-06     97677.118
2021-06-06     97090.846
2021-04-06     96143.819
2021-11-06     95145.299
2021-05-29     94360.049
2021-06-19     94032.322
2021-06-26     93509.955
Name: PEATONES, dtype: float64

In [ ]:
pedestrians.to_csv('peatones_limpios.csv')

In [25]:
idealista = pd.read_excel('/Users/Usuario/demo/Ironhack/Final_Project/raw_data/public_data/idealista.xlsx')
idealista.head()

,Localización,Precio m2 ene 2023,Variación mensual,Variación trimestral,Variación anual,Máximo histórico,Variación máximo
0,Madrid,"16,5 €/m2",0.020,0.013,0.123,"16,8 €/m2 mayo 2020",-0.015
1,Arganzuela,"16,3 €/m2",0.009,0.026,0.105,"16,4 €/m2 mayo 2020",-0.011
2,Barajas,"12,6 €/m2",0.010,0.035,0.085,"12,6 €/m2 ene 2023",0.000
3,Carabanchel,"12,6 €/m2",-0.003,0.011,0.082,"12,7 €/m2 jun 2020",-0.006
4,Centro,"21,1 €/m2",0.041,0.090,0.184,"21,1 €/m2 ene 2023",0.000


In [26]:
idealista.shape

(22, 7)

In [27]:
idealista

,Localización,Precio m2 ene 2023,Variación mensual,Variación trimestral,Variación anual,Máximo histórico,Variación máximo
0,Madrid,"16,5 €/m2",0.020,0.013,0.123,"16,8 €/m2 mayo 2020",-0.015
1,Arganzuela,"16,3 €/m2",0.009,0.026,0.105,"16,4 €/m2 mayo 2020",-0.011
2,Barajas,"12,6 €/m2",0.010,0.035,0.085,"12,6 €/m2 ene 2023",0.000
3,Carabanchel,"12,6 €/m2",-0.003,0.011,0.082,"12,7 €/m2 jun 2020",-0.006
4,Centro,"21,1 €/m2",0.041,0.090,0.184,"21,1 €/m2 ene 2023",0.000
5,Chamartín,"17,2 €/m2",0.022,0.028,0.119,"17,2 €/m2 ene 2023",0.000
6,Chamberí,"20,0 €/m2",0.014,0.072,0.164,"20,0 €/m2 ene 2023",0.000
7,Ciudad Lineal,"14,3 €/m2",0.005,0.037,0.105,"14,3 €/m2 ene 2023",0.000
8,Fuencarral,"13,2 €/m2",0.009,0.015,0.064,"13,2 €/m2 ene 2023",0.000
9,Hortaleza,"13,5 €/m2",0.015,0.024,0.072,"13,5 €/m2 ene 2023",0.000


In [28]:
def normalize(s):
    replacements = (
        ("á", "a"),
        ("é", "e"),
        ("í", "i"),
        ("ó", "o"),
        ("ú", "u"),
    )
    for a, b in replacements:
        s = s.replace(a, b).replace(a.upper(), b.upper())
    return s

In [31]:
idealista['Localización'] = idealista['Localización'].apply(normalize)
idealista['Localización'] = idealista['Localización'].str.upper()
idealista['Localización'] = idealista['Localización'].str.strip()

In [35]:
import re
def remove_euro(text):
    return re.sub(r'\s*€/m2', '', str(text))

In [36]:
idealista['Precio m2 ene 2023'] = idealista['Precio m2 ene 2023'].apply(remove_euro)

In [40]:
idealista['Precio m2 ene 2023'] = idealista['Precio m2 ene 2023'].str.replace(",",".")

In [42]:
idealista['Precio m2 ene 2023'] = idealista['Precio m2 ene 2023'].astype(float)

In [43]:
idealista.describe()

,Precio m2 ene 2023,Variación mensual,Variación trimestral,Variación anual,Variación máximo
count,22.000000,22.000000,22.000000,22.000000,22.000000
mean,14.845455,0.011818,0.024364,0.107045,-0.004909
std,2.965312,0.012606,0.024632,0.037276,0.009086
min,11.300000,-0.009000,-0.036000,0.064000,-0.036000
25%,12.600000,0.005000,0.011500,0.082500,-0.005500
50%,13.400000,0.009500,0.023500,0.095000,0.000000
75%,16.800000,0.019500,0.032750,0.122000,0.000000
max,21.100000,0.041000,0.090000,0.184000,0.000000


In [46]:
idealista_columns = ['District', 'Price_squared_meter', 'Monthly var',
       'Quarterly var', 'Yearly var', 'Máximo histórico',
       'Variación máximo']
idealista.columns= idealista_columns

In [48]:
idealista.drop(columns=['Máximo histórico','Variación máximo'], axis=1, inplace=True)

In [51]:
idealista = idealista.drop(idealista.index[0])
idealista.head()

,District,Price_squared_meter,Monthly var,Quarterly var,Yearly var
1,ARGANZUELA,16.3,0.009,0.026,0.105
2,BARAJAS,12.6,0.010,0.035,0.085
3,CARABANCHEL,12.6,-0.003,0.011,0.082
4,CENTRO,21.1,0.041,0.090,0.184
5,CHAMARTIN,17.2,0.022,0.028,0.119


In [52]:
idealista.to_csv('idealista.csv')

In [1]:
import pandas as pd
coo = pd.read_csv('coordinates.csv')
coo

,Unnamed: 0,result.address_components,result.business_status,result.current_opening_hours.periods,result.current_opening_hours.weekday_text,result.delivery,result.dine_in,result.formatted_address,result.formatted_phone_number,result.geometry.location.lat,...,result.serves_wine,result.takeout,result.types,result.url,result.user_ratings_total,result.utc_offset,result.vicinity,result.website,result.wheelchair_accessible_entrance,error_message
0,0,"[{'long_name': '10', 'short_name': '10', 'type...",OPERATIONAL,"[{'close': {'date': '2023-02-27', 'day': 1, 't...",['Monday: 12:30\u202fPM\u2009–\u200912:00\u202...,True,True,"C. San Juan de Ortega, 10, 28050 Madrid, Spain",913 07 52 88,40.502148,...,True,True,"['restaurant', 'food', 'point_of_interest', 'e...",https://maps.google.com/?cid=13282398378271868427,1317.0,60.0,"Calle San Juan de Ortega, 10, Madrid",http://www.grupoladehesa.com/,True,NaN
1,1,"[{'long_name': '1', 'short_name': '1', 'types'...",OPERATIONAL,"[{'close': {'date': '2023-02-26', 'day': 0, 't...",['Monday: 10:00\u202fAM\u2009–\u200910:00\u202...,True,NaN,"C. de Margarita de Parma, 1, 28050 Madrid, Spain",913 84 82 00,40.497487,...,NaN,NaN,"['shopping_mall', 'cafe', 'meal_delivery', 'pa...",https://maps.google.com/?cid=3737344296213236355,17623.0,60.0,"Calle de Margarita de Parma, 1, Madrid",https://www.elcorteingles.es/centroscomerciale...,True,NaN
2,2,"[{'long_name': 'Zona', 'short_name': 'Zona', '...",OPERATIONAL,"[{'close': {'date': '2023-02-26', 'day': 0, 't...","['Monday: Closed', 'Tuesday: 12:00\u2009–\u200...",True,True,"Paseo de la, Zona, P.º de la Tierra de Melide,...",917 50 53 93,40.511201,...,True,True,"['restaurant', 'food', 'point_of_interest', 'e...",https://maps.google.com/?cid=14070834893686348892,602.0,60.0,"Zona, Paseo de la, Paseo de la Tierra de Melid...",https://www.casacorrochano.es/,NaN,NaN
3,3,"[{'long_name': 'Plaza de la Moraleja', 'short_...",OPERATIONAL,"[{'close': {'date': '2023-02-27', 'day': 1, 't...",['Monday: 1:00\u202fPM\u2009–\u200912:00\u202f...,False,True,"Pl. de la Moraleja, 28109 Alcobendas, Madrid, ...",916 25 25 00,40.515106,...,True,True,"['restaurant', 'food', 'point_of_interest', 'e...",https://maps.google.com/?cid=16618770644644484753,595.0,60.0,"Plaza de la Moraleja, Alcobendas",https://restauranteaspen.com/,True,NaN
4,4,"[{'long_name': '1', 'short_name': '1', 'types'...",OPERATIONAL,"[{'close': {'date': '2023-02-27', 'day': 1, 't...",['Monday: 12:30\u202fPM\u2009–\u20092:00\u202f...,False,True,"Pl. de la Moraleja, 1, 28109 Alcobendas, Madri...",916 58 52 97,40.515084,...,True,True,"['restaurant', 'food', 'point_of_interest', 'e...",https://maps.google.com/?cid=5931455114014290729,2390.0,60.0,"Plaza de la Moraleja, 1, Alcobendas",https://lamaquinalamoraleja.es/,True,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6303,6303,"[{'long_name': '3', 'short_name': '3', 'types'...",CLOSED_TEMPORARILY,NaN,NaN,NaN,True,"C. de Sepúlveda, 3, 28011 Madrid, Spain",NaN,40.406758,...,NaN,NaN,"['restaurant', 'food', 'point_of_interest', 'e...",https://maps.google.com/?cid=17812421101481639147,NaN,60.0,"Calle de Sepúlveda, 3, Madrid",NaN,NaN,NaN
6304,6304,"[{'long_name': '48', 'short_name': '48', 'type...",OPERATIONAL,NaN,NaN,False,True,"P.º de la Ermita del Santo, 48, 28011 Madrid, ...",NaN,40.406880,...,True,True,"['restaurant', 'food', 'point_of_interest', 'e...",https://maps.google.com/?cid=5107148488018157553,2.0,60.0,"Paseo de la Ermita del Santo, 48, Madrid",NaN,NaN,NaN
6305,6305,"[{'long_name': '3', 'short_name': '3', 'types'...",OPERATIONAL,"[{'close': {'date': '2023-02-26', 'day': 0, 't...","['Monday: 2:00\u2009–\u20098:00\u202fPM', 'Tue...",False,True,"C. de Sepúlveda, 3, 28011 Madrid, Spain",663 51 23 67,40.407050,...,True,True,"['restaurant', 'liquor_store', 'cafe', 'bar', ...",https://maps.google.com/?cid=6783403310523868511,131.0,60.0,"Calle de Sepúlveda, 3, Madrid",https://restaurante-tuc-tuc.eatbu.com/,True,NaN
6306,6306,"[{'long

In [3]:
coo.drop(columns=['Unnamed: 0', 'result.address_components', 'result.business_status',
       'result.current_opening_hours.periods',
       'result.current_opening_hours.weekday_text', 'result.delivery',
       'result.dine_in', 'result.formatted_address',
       'result.formatted_phone_number',
       'result.international_phone_number',
       'result.opening_hours.open_now', 'result.opening_hours.periods',
       'result.opening_hours.weekday_text', 'result.photos',
       'result.plus_code.compound_code', 'result.plus_code.global_code',
       'result.price_level', 'result.rating', 'result.reference',
       'result.reservable', 'result.reviews', 'result.serves_beer',
       'result.serves_breakfast', 'result.serves_brunch',
       'result.serves_dinner', 'result.serves_lunch',
       'result.serves_vegetarian_food', 'result.serves_wine', 'result.takeout',
       'result.types', 'result.url', 'result.user_ratings_total',
       'result.utc_offset', 'result.vicinity', 'result.website',
       'result.wheelchair_accessible_entrance', 'error_message'], axis=1, inplace=True)

In [5]:
coo.to_csv('coordinates.csv', index=False)